In [1]:
import os
import openai
import json
import random
import time
import re
openai.api_type = "azure"
openai.api_base = "https://test-openai-celesh.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = "24a7f6c60aed4aae9c9729229709857b"

In [2]:
inst_prompt = """\"\"\"
{note}
\"\"\"

Using above patient's discharge summary, you want to ask a well-trained model to help your clinical decision making.
Below are high-level NLP categories that you might want to ask about.
Tell me {num_inst_per_note} different instructions that you will ask to the model.

Tasks :
{tasks}

Here are requirements:
1. The words used in the instructions should be diverse to maximize the diversity.
2. A GPT language model should be able to complete the instruction. For example, do not ask the model to create other than any textual output.
3. The question type should be diverse such as one-word answer question, open-ended question, multiple-choice question, and yes/no question.
4. Concat the used NLP categories on front the questions such as "" (Text Classification) 'The question' "".
"""

ans_prompt = """\"\"\"
{note}
\"\"\"

Using above patient's discharge summary, answer to the following questions.
The question is given with their own categoreis, with colon-concatenated form.

Questions:
{questions}

Here are requirements:

1. Note that each question is independent.
2. Number your responses.
3. Ensure that each answer is complete and does not raise another question.
4. Answers can span multiple lines if needed.
5. Answer all questions!
6. Standardize the terminology used to describe the patient's condition?

format the output in MARKDOWN format：

- "qas"
  - [i]
    - "answers"
      - [j]
        - "answer_start"
        - "text"
    - "id" 
    - "question"


The top-level is "qas", which has a value that is a list of elements, denoted by [i]. Each element in the list is an object with three parts: "answers", "id", and "question".

The "answers" has a value that is a list of elements, denoted by [j]. Each element in the list is an object with two parts: "answer_start" and "text".

Important: If there are multiple answers or the answers have multiple sections or the answers have multiple sentences in parallel such as `1..., 2..., 3...` ,`A..., B...,C....`,`1)..., 2)..., 3)....` and so on, make sure to divide the whole answer into multiple "answer_start"-"text" pairs under the "answers" part, such as

- "answers"
  - [0]
    - "answer_start"
    - "text": section 1 of the answer or answer 1
  - [1]
    - "answer_start"
    - "text": section 2 of the answer or answer 2
  - [2]
    - "answer_start"
    - "text": section 3 of the answer or answer 3

The "answer_start" has a value representing the index of the first character of the answer string in the context. 

The "text" has a value representing the answer text which contain some information and can't be empty.

The "id" has a value representing the identifier of the question. It can be found in the begining of each question.

"question": This is a string representing the question being asked. Just copy the input question under this part. Include the NLP categories on front in this format "" (Text Classification) 'The question' "".

"""

tasks = [
    "Question Answering",
    "Natural Language Generation",
    "Text Summarization",
    "Text Classification",
    "Temporal Information Extraction",
    "Relation Extraction",
    "Named Entity Recognition",
    "Paraphrasing",
    "Clinical Concept Normalization",
    "Keyword Extraction",
    "Abbreviation expansion",
    "Feature extraction",
    "Content extraction"
]

In [3]:
question_list = [
    "(Text Summarization) What is the patient's primary diagnosis? If the answers have multiple sections in parallel, make sure to divide the whole answer into multiple answer_start-text pairs under the answers part.",
    
    "(Content extraction) Which sentences in the note describe the diagnosis?",
    
    "(Text Summarization) What are the comorbidities of the patient? If the answers have multiple sections in parallel, make sure to divide the whole answer into multiple answer_start-text pairs under the answers part.",
    
    "(Content extraction) Which sentences in the note describe the comorbidities?",
    
    "(Named Entity Recognition) List all the medications mentioned in the discharge summary. If the answers have multiple sections in parallel, make sure to divide the whole answer into multiple answer_start-text pairs under the answers part.",
    
    "(Content extraction) Which sentences in the note describe the medications?",

    "(Named Entity Recognition) List all the labs mentioned in the discharge summary. If the answers have multiple sections in parallel, make sure to divide the whole answer into multiple answer_start-text pairs under the answers part.",

    "(Content extraction) Which sentences in the note describe the labs?",
    
    "(Text Summarization) Provide a brief summary of the history treatment plan for the patient?",

    "(Content extraction) Which sentences in the note describe the history treatment plan for the patient?",
    
    "(Text Summarization) Provide a brief summary of the future treatment plan for the patient?",

    "(Content extraction) Which sentences in the note describe the future treatment plan for the patient?",
    
    "(Feature extraction) What medications are the patient currently on? If the answers have multiple sections in parallel, make sure to divide the whole answer into multiple answer_start-text pairs under the answers part.",
    
    "(Text Summarization) List all the diagnoses and determine whether they are from recent onsite visits or past medical history.  If the answers have multiple sections in parallel, make sure to divide the whole answer into multiple answer_start-text pairs under the answers part."
]
print(len(question_list))

14


In [4]:
def generate_inst_prompt(note, num_inst_per_note, sample = True):
    if sample:
        selected_tasks = random.sample(tasks, num_inst_per_note)
    else: selected_tasks = tasks
    
    return [{"role":"system","content":"You are a healthcare professional."},
        {
            "role": "user",
            "content": inst_prompt.format(
                note=note,
                num_inst_per_note=num_inst_per_note,
                tasks="\n".join(selected_tasks),
            ),
        }
    ]

def generate_ans_prompt(note, questions):
    return [{"role":"system","content":"You are a healthcare professional."},
        {"role": "user", "content": ans_prompt.format(note=note, questions=questions)}
    ]

def get_completion(message,temperature=0.6):
    num_tries = 0
    max_num_tries = 10

    while num_tries < max_num_tries:
        try:
            print("try get_completion...")
            response = openai.ChatCompletion.create(
                engine="chat-test",
                messages = message,
                temperature=temperature,
                max_tokens=4096,
                top_p=0.95,
                frequency_penalty=0,
                presence_penalty=0,
                stop=None)

        except:
            num_tries += 1
            print(f"Error occurred in get_completion(), retrying (try {num_tries})...")
            time.sleep(5)
            continue
        
        print(f"Success! get_completion() completed. tried {num_tries}")     
        break
    
    if num_tries == max_num_tries:
        print("⚠Error: maximum number of tries reached in function get_completion().") 
        return " "
     
    return response["choices"][0]["message"]["content"]



In [5]:
get_completion([{"role": "user", "content": "introduce harvard"}])

try get_completion...
Success! get_completion() completed. tried 0


"Harvard University is a private Ivy League research university located in Cambridge, Massachusetts, United States. Established in 1636 and named after its first benefactor, clergyman John Harvard, Harvard is the oldest institution of higher learning in the United States and among the most prestigious in the world. The Massachusetts colonial legislature, the General Court, authorized Harvard's founding. In its early years, Harvard College primarily trained Congregational and Unitarian clergy, although it has never been formally affiliated with any denomination. Its curriculum and student body were gradually secularized during the 18th century, and by the 19th century, Harvard had emerged as the central cultural establishment among Boston elites. Following the American Civil War, President Charles W. Eliot's long tenure (1869–1909) transformed the college and affiliated professional schools into a modern research university; Harvard became a founding member of the Association of America

In [6]:
import pandas as pd
df = pd.read_csv('mimiciv_discharge.csv')
df = df.loc[[19,18,17],].reset_index(drop=True)

In [7]:
from transformers import LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-13b-hf")

length = df['text'].apply(lambda x: len(tokenizer.tokenize(x)))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [8]:
length

0    2038
1    2586
2    6227
Name: text, dtype: int64

In [9]:
def process_text(text,threshold=0.05):
    
    ## delete \n
    text = text.replace('\n', '')
    
    ##delete the useless part
    match = re.search(r'(?<=)Sex.*?(?=Followup Instructions:)', text)
    text = match.group(0).strip() if match else text
    
    ## split the text
    sections = re.split(r'([A-Z\s]+:)', text)
    underscore_ratios = [section.count('_') / len(section) if len(section) > 0 else 0 for section in sections]
    sections = [section for section, ratio in zip(sections, underscore_ratios) if ratio <= threshold]
    text = ''.join(sections)
    
    return text

df['text'] = df['text'].apply(process_text)

In [10]:
def find_last_number(text):
    last_number = 0
    for i in re.findall(r'\d+\.', text):
        if int(i[:-1]) > last_number:
            last_number = int(i[:-1])
    return last_number

def add_question_list(questions, question_list):
    num = len(question_list);temp = ""
    for i in range(len(question_list)):
        temp += str(i + 1) + ". " + question_list[i]+ "\n"
    questions = temp + add_numbers(questions,num)
    return questions

def split_questions(questions,m):
    question_list = re.findall(r'\d+\.\s+[^\d]+', questions)
    question_str = ""
    question_str_list = []
    for i in range(len(question_list)):
        if i % m == 0 and i != 0:
            question_str_list.append(question_str)
            question_str = ""
        question_str += question_list[i] + "\n"
    question_str_list.append(question_str)
    return question_str_list

def find_substring(string):
    start = string.find("{")
    end = string.rfind("}")
    return string[start:end+1]

def add_numbers(string,num):
    pattern = r'\b(\d+)\b'
    replacement = lambda match: str(int(match.group(1)) + num)
    result = re.sub(pattern, replacement, string)
    return result

In [11]:
data = {}
data["version"] = "mimic4"
data["data"] = []

for row in range(3):
    time.sleep(5)
    print("========================================= row:", row, "=========================================")
    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    note = df.iloc[row,-1]
    
    n = 16
    question_message = generate_inst_prompt(note, num_inst_per_note = n, sample = False)
    questions = get_completion(question_message)

    
    questions = add_question_list(questions, question_list)
    
    print("```")
    print(questions)
    print("```")
    
    m = 10
    question_str_list = split_questions(questions, m)
    # print(len(question_str_list))
    
    new_data = {}
    new_data["title"] = df.iloc[row,0]
    new_data["paragraphs"] = [{"context": df.iloc[row,-1], "qas": []}]

    for question_str in (question_str_list):
        
        num_tries = 0
        max_num_tries = 5

        while num_tries < max_num_tries:
            try:
                answer_message = generate_ans_prompt(note, question_str)
                answers = get_completion(answer_message)
                
                print("-----------------------------------------")
                print(answers)
                print("-----------------------------------------")
                
                new_qas = json.loads(answers)

            except:
                num_tries += 1
                print("-----------------------------------------")
                print(f"    Error occurred in json.loads(), retrying (try {num_tries})...")
                print("-----------------------------------------")
                print("-----------------------------------------")
                print(answers)
                print("-----------------------------------------")
                continue
                
            print(f"    Success! json.loads() completed. tried {num_tries}")
            for foo in new_qas["qas"]:
                new_data["paragraphs"][0]["qas"].append(foo)
            
            print(len(new_data["paragraphs"][0]["qas"]))
            if len(new_qas["qas"]) != m:
                print(question_str)
                print(answers)
            break
        
        
        if num_tries == max_num_tries:
            print("    ⚠Error: maximum number of tries reached in function json.loads().")
            print(question_str)
            print(answers)
        
    data["data"].append(new_data)
    print(len(new_data['paragraphs'][0]['qas']))
    print("\n")
    # print(data)
    with open('tmp5.json', 'w') as f:
        json.dump(data, f, indent=2)


========================================= row: 0 =========================================
2023-05-23 21:22:31
try get_completion...
Success! get_completion() completed. tried 0
```
1. (Text Summarization) What is the patient's primary diagnosis? If the answers have multiple sections in parallel, make sure to divide the whole answer into multiple answer_start-text pairs under the answers key.
2. (Content extraction) Which sentences in the note describe the diagnosis?
3. (Text Summarization) What are the comorbidities of the patient? If the answers have multiple sections in parallel, make sure to divide the whole answer into multiple answer_start-text pairs under the answers key.
4. (Content extraction) Which sentences in the note describe the comorbidities?
5. (Named Entity Recognition) List all the medications mentioned in the discharge summary. If the answers have multiple sections in parallel, make sure to divide the whole answer into multiple answer_start-text pairs under the answe

In [ ]:
note = df.text[0]
answer_message = generate_ans_prompt(note, question_list[1])

In [ ]:
response = openai.ChatCompletion.create(
    engine="chat-test",
    messages = answer_message,
    temperature=0.65,
    max_tokens=4096,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)

In [ ]:
data

{'version': 'mimic4',
 'data': [{'title': '10071869-DS-19',
   'paragraphs': [{'context': 5974,
     'qas': [{'id': 1,
       'question': "(Text Summarization) What is the patient's primary diagnosis?",
       'answers': [{'answer_start': 0,
         'text': 'Patient with a history of mitral regurgitation and hypertension, now with acute decompensated heart failure.'}]},
      {'id': 2,
       'question': '(Content extraction) Which sentences in the note describe the diagnosis?',
       'answers': [{'answer_start': 0,
         'text': 'Admitted due to shortness of breath, found to have acute decompensated heart failure with reduced ejection fraction. Patient with a history of mitral regurgitation and hypertension.'}]},
      {'id': 3,
       'question': '(Text Summarization) What are the comorbidities of the patient?',
       'answers': [{'answer_start': 73,
         'text': 'Patient with a history of mitral regurgitation and hypertension.'}]},
      {'id': 4,
       'question': '(Cont

In [ ]:
df.iloc[row]

note_id                                          10071869-DS-19
subject_id                                             10071869
hadm_id                                                20895291
note_type                                                    DS
note_seq                                                     19
charttime                                   2111-01-12 00:00:00
storetime                                   2111-01-12 02:41:00
text           Name:  ___                     Unit No:   ___...
length                                                     5974
Name: 0, dtype: object